In [ ]:
#| default_exp fastmcp_tools
#| test: false

#"""Shared FastMCP SSE server exposing tools."""


In [ ]:
#| export
#| eval: false

import logging, itertools, json, re
from functools import lru_cache
from dataclasses import dataclass, field
from typing import List, Dict, Any
from DataTalks.api_clients import Mode as Route_Mode
import httpx, yaml
from fastmcp import FastMCP  
from pydantic import ValidationError 
import os, yaml, httpx
from fastmcp import FastMCP
import requests
from requests.structures import CaseInsensitiveDict
import os
import json
import httpx
from fastmcp import FastMCP
from fastmcp.server.openapi import RouteMap, MCPType



from DataTalks.parking_tools import (
    nearest_parking_ids,
    parking_status,
    facility_prediction,
)

from DataTalks.api_clients import (
    Point,
    osrm_car,
    osrm_walk,
    osrm_bike,
    digitransit_pt,
    Route
)

from DataTalks.api_clients import Point
from fastmcp.server.openapi import RouteMap, MCPType
import httpx, yaml, os
from fastmcp import FastMCP
from typing import Literal


ImportError: cannot import name 'MCPType' from 'fastmcp.server.openapi' (/home/nata/miniconda3/envs/datatalks/lib/python3.11/site-packages/fastmcp/server/openapi.py)

In [ ]:
#| export
#| eval: false

ROUTING_SPEC_URL = (
    "https://raw.githubusercontent.com/geoapify/geoapify-openapi-specs/main/"
    "api-specs/routing/routing.yaml"
)
GEOAPIFY_KEY = os.environ["GEOAPIFY_KEY"]

_HKEY = os.environ["HOCKEY_KEY"]       
_HOCKEY_SPEC_PATH = "hockey_openapi.json"
_BASE_URL = "https://hockey-highlights-api.p.rapidapi.com"


KeyError: 'GEOAPIFY_KEY'

In [ ]:
#| export
#| eval: false

mcp = FastMCP(name="tools")
app = mcp.http_app(transport="sse", path="/sse")

In [ ]:
#| export
#| eval: false

@app.on_event("startup")
async def _auto_mount_tools() -> None:
    try:
        tools = await _mount_hockey_api(mcp, name="hockey")
        logging.info("✅ Hockey API mounted with %s tools: %s", len(tools), tools)
    except Exception as exc:
        logging.error("❌ Could not mount Hockey API: %s", exc)
    try:
        tools = await _mount_weather_api(mcp, name="weather")
        logging.info("✅ Weather API mounted with %s tools: %s", len(tools), tools)
    except Exception as exc:
        logging.error("❌ Could not mount Weather API: %s", exc)     
        
#    Client request for  geoapify not established.       
#    try:
#        tools = await _mount_routing_api(mcp, name="geoapify_routing")
#        logging.info("✅ GeoApiFy API mounted with %s tools: %s", len(tools), tools)
#    except Exception as exc:
#        logging.error("❌ Could not mount GeoApiFy API: %s", exc)    
#    try:
#        tools = await _mount_geocoding_api(mcp, name="geoapify_geocoding")
#        logging.info("✅ GeoApiFy API mounted with %s tools: %s", len(tools), tools)
#    except Exception as exc:
#        logging.error("❌ Could not mount GeoApiFy API: %s", exc)      

In [ ]:
#| export
#| eval: false

async def _mount_hockey_api(parent: FastMCP, name: str = "hockey") -> list[str]:
    """
    Mount only the GET /highlights/{id} route from the hockey API.
    All other routes are excluded.
    """
    with open(_HOCKEY_SPEC_PATH, "r", encoding="utf-8") as fp:
        spec = json.load(fp)

    # HTTP client that always sends your rapidapi key
    client = httpx.AsyncClient(
        base_url=_BASE_URL,
        headers={
            "x-rapidapi-key": _HKEY,
            "x-rapidapi-host": "_hockey-highlights-api.p.rapidapi.com", ###!!!!!_
        },
        timeout=120.0,
    )

    # RouteMap so that only GET /highlights/{id} is included → tool
    # Everything else is excluded
    route_maps = [
        RouteMap(
            methods=["GET"],
            pattern=r"^/highlights/\{.*\}",  # match /highlights/{something}
            mcp_type=MCPType.TOOL
        ),
        RouteMap(mcp_type=MCPType.EXCLUDE),
    ]

    sub = FastMCP.from_openapi(
        openapi_spec=spec,
        client=client,
        name=name,
        route_maps=route_maps
    )

    parent.mount(name, sub)  # expose as /hockey/...
    return [t.name for t in (await sub.get_tools()).values()]



In [ ]:
#| export
#| eval: false

from anyio import Path


async def _mount_routing_api(parent: FastMCP,
                             name: str = "geoapify_routing") -> list[str]:
    
    
    #spec_txt = (await httpx.AsyncClient(timeout=30)
    #            .get("https://raw.githubusercontent.com/geoapify/"
    #                 "geoapify-openapi-specs/main/api-specs/routing/routing.yaml")).text
    
    yaml_text = await Path("geoapify_routing_openapi_spec.yaml").read_text()
    
    spec      = yaml.safe_load(yaml_text)

    api_client = httpx.AsyncClient(
        base_url="https://api.geoapify.com/v1",
        params={"apiKey": "233f85aca300415c86dff68a22af6b53"},               # auto-inject key
        #headers=headers,
        timeout=120,
    )
    
    api_client.headers.update({
        "Accept": "application/json",
        "User-Agent": "MyApp/1.0 (+you@yourdomain.com)",  # ← must send a real UA
        "Accept-Language": "en-US,en;q=0.9",
    })

    route_maps = [
        RouteMap(methods=["GET"], pattern=r"^/routing/?(?:\?.*)?$", mcp_type=MCPType.TOOL), 
        RouteMap(mcp_type=MCPType.EXCLUDE), ]

    sub = FastMCP.from_openapi(spec, api_client, name=name,route_maps=route_maps)
    parent.mount(name, sub)
    return [t.name for t in (await sub.get_tools()).values()]



In [ ]:
#| export
#| eval: false

from anyio import Path


async def _mount_geocoding_api(parent: FastMCP,
                             name: str = "geoapify_geocoding") -> list[str]:
    
    
    #spec_txt = (await httpx.AsyncClient(timeout=30)
    #            .get("https://raw.githubusercontent.com/geoapify/"
    #                 "geoapify-openapi-specs/main/api-specs/routing/routing.yaml")).text
    
    yaml_text = await Path("geoapify_geocoding_openapi_spec.yaml").read_text()
    
    spec      = yaml.safe_load(yaml_text)



    api_client = httpx.AsyncClient(
        base_url="https://api.geoapify.com/v1",
        params={"apiKey": "233f85aca300415c86dff68a22af6b53"},               # auto-inject key
        #headers=headers,
        timeout=30,
    )
    api_client.headers.update({
        "Accept": "application/json",
        "User-Agent": "MyApp/1.0 (+you@yourdomain.com)",  # ← must send a real UA
        "Accept-Language": "en-US,en;q=0.9",
    })

    route_maps = [
        RouteMap(methods=["GET"],pattern=r"^/geocode/(search|autocomplete|reverse)/?(?:\?.*)?$", mcp_type=MCPType.TOOL),       
        RouteMap(mcp_type=MCPType.EXCLUDE),
    ]

    sub = FastMCP.from_openapi(spec, api_client, name=name,route_maps=route_maps)
    parent.mount(name, sub)
    return [t.name for t in (await sub.get_tools()).values()]

In [ ]:
#| export
#| eval: false

import os, requests
from urllib.parse import quote_plus
from typing import List, Tuple
from requests.structures import CaseInsensitiveDict

BASE_URL = "https://api.geoapify.com/v1"
API_KEY  = os.getenv("GEOAPIFY_KEY", "").strip()
MAPS_URL    = "https://maps.geoapify.com/v1/staticmap"

if not API_KEY:
    raise RuntimeError("GEOAPIFY_KEY env-var is missing")

_HEADERS = CaseInsensitiveDict({"Accept": "application/json"})

# ────────── tiny helpers ────────────────────────────────────────────────────
def _extract_latlon(payload: dict) -> Tuple[float, float]:
    """Return (lat, lon) for either `results` (JSON) or `features` (GeoJSON)."""
    if payload.get("results"):       # format=json
        hit = payload["results"][0]
        return float(hit["lat"]), float(hit["lon"])
    if payload.get("features"):      # format=geojson
        props = payload["features"][0]["properties"]
        return float(props["lat"]), float(props["lon"])
    raise ValueError("no geocoding hits in response")

def _fwd_geocode(addr: str) -> Tuple[float, float]:
    url = (
        f"{BASE_URL}/geocode/search?"
        f"text={quote_plus(addr)}&limit=1&format=json&apiKey={API_KEY}"
    )
    logging.info("✅ URL FOR FWD GEOCODING %s", url)
    r = requests.get(url, headers=_HEADERS, timeout=30)
    status = r.raise_for_status()
    logging.info("✅ FWD_GEOCODE STATUS %s", status)
    return _extract_latlon(r.json())

def _rev_geocode(lat: float, lon: float) -> str:
    url = (
        f"{BASE_URL}/geocode/reverse?"
        f"lat={lat}&lon={lon}&limit=1&format=json&apiKey={API_KEY}"
    )
    r = requests.get(url, headers=_HEADERS, timeout=30)
    r.raise_for_status()
    j = r.json()
    if j.get("results"):
        return j["results"][0]["formatted"]
    if j.get("features"):
        return j["features"][0]["properties"].get("formatted")
    return f"{lat:.5f},{lon:.5f}"

# ── 1. flatten any geometry shape into lon/lat pairs ─────────────────────
def _flatten_geometry(geom: object) -> list[tuple[float, float]]:
    """
    Works for every current Geoapify shape:
      • JSON  : list[list[{'lon':..,'lat':..}]]
      • JSON2 : list[list[[lon,lat] or [lon,lat,z]]]
      • GeoJSON MultiLineString
    """
    pts: list[tuple[float, float]] = []

    if isinstance(geom, dict) and "coordinates" in geom:          # GeoJSON
        geom = geom["coordinates"]

    if isinstance(geom, list):
        for segment in geom:
            if isinstance(segment, dict):                         # {lon,lat}
                pts.append((segment["lon"], segment["lat"]))
            elif isinstance(segment, list):
                # segment could be a list of dicts or of coordinate arrays
                for p in segment:
                    if isinstance(p, dict):                       # {lon,lat}
                        pts.append((p["lon"], p["lat"]))
                    elif isinstance(p, (list, tuple)):
                        # [lon, lat] or [lon, lat, …]
                        if len(p) >= 2:
                            pts.append((float(p[0]), float(p[1])))
    return pts

def _step_text(step: dict) -> str:
    """Spec-compliant extraction of human instruction text."""
    instr = step.get("instruction")
    if isinstance(instr, dict):   # instruction_details requested
        return instr.get("text") or str(instr)
    return str(instr)             # plain string fallback

# ────────── core worker ────────────────────────────────────────────────────
from typing import TypedDict, List, Tuple

Coord = Tuple[float, float]          # (lon, lat)


class RouteResult(TypedDict):
    coords: List[Coord]              # decoded polyline
    steps:  List[str]                # already prettified texts


def _route(start: str, finish: str, mode: str) -> RouteResult:
    s_lat, s_lon = _fwd_geocode(start)
    f_lat, f_lon = _fwd_geocode(finish)
    waypoints    = f"{s_lat},{s_lon}|{f_lat},{f_lon}"
    #waypoints = f"{s_lon},{s_lat}|{f_lon},{f_lat}"

    url = (
        f"{BASE_URL}/routing?"
        f"waypoints={quote_plus(waypoints)}"
        f"&mode={mode}"
        f"&details=instruction_details,route_details"
        f"&format=json"
        f"&apiKey={API_KEY}"
    )
    r = requests.get(url, headers=_HEADERS, timeout=60)
    r.raise_for_status()
    data = r.json()

    if "results" in data:
        route = data["results"][0]
        geom  = route["geometry"]
        legs  = route["legs"]
    elif "features" in data:
        feat  = data["features"][0]
        geom  = feat["geometry"]
        legs  = feat["properties"]["legs"]
    else:
        raise KeyError("Geoapify response lacked 'results' or 'features'.")

    coords = _flatten_geometry(geom)               # List[Coord]
    step_txt = [
        f"{_step_text(st)}  ➜  {_rev_geocode(*coords[st['from_index']][::-1])}"
        for st in legs[0]["steps"]
    ]
    return {"coords": coords, "steps": step_txt}


from urllib.parse import quote_plus

from urllib.parse import quote_plus

def _build_static_map_url(coords: list[tuple[float, float]]) -> str:
    """
    Build a Static-Map URL that Geoapify accepts (geometry=polyline:…).
    *No* encoding step means shorter but longer URLs; fine for <2 k points.
    """
    # keep only lon,lat (drop elevation/time if present)
    clean = [(float(pt[0]), float(pt[1])) for pt in coords if len(pt) >= 2]

    # polyline:lon1,lat1,lon2,lat2…
    coord_str = ",".join(f"{lon:.6f},{lat:.6f}" for lon, lat in clean)
    geom      = f"polyline:{coord_str}"                  # plain-text polyline
    geom_q    = quote_plus(geom, safe=":,")              # encode ',' ':' but keep them

    return (
        "https://maps.geoapify.com/v1/staticmap"
        f"?apiKey={API_KEY}"
        "&style=osm-bright"
        "&width=600&height=400"
        f"&geometry={geom_q}"                            # ← FIXED
    )



# ────────── mcp.tool wrappers (one GET each, no sessions) ──────────────────
@mcp.tool("route_car")
def route_car(start: str, finish: str) -> str:
    """Turn-by-turn for motorists, returned as a static PNG map."""
    result = _route(start, finish, mode="drive")
    return _build_static_map_url(result["coords"])


@mcp.tool("route_bike")
def route_bike(start: str, finish: str) -> str:
    """Cyclist-friendly routing as a static map."""
    result = _route(start, finish, mode="bicycle")
    return _build_static_map_url(result["coords"])


@mcp.tool("route_walk")
def route_walk(start: str, finish: str) -> str:
    result   = _route(start, finish, mode="walk")   # RouteResult dataclass
    return _build_static_map_url(result.coords)     # .coords  NOT ["coords"]



@mcp.tool("route_public_transport")
def route_public_transport(start: str, finish: str) -> List[str]:
    """Step-by-step public-transport itinerary (text only)."""
    return _route(start, finish, mode="transit")["steps"]




RuntimeError: GEOAPIFY_KEY env-var is missing

In [ ]:
#| export
#| eval: false

async def _mount_weather_api(parent: FastMCP, name: str = "weather") -> list[str]:
    from anyio import Path

    # ➊ load the spec that you shipped beside the code
    yaml_text    = await Path("weather_openapi_spec.yaml").read_text()
    openapi_spec = yaml.safe_load(yaml_text)

    # ➋ HTTP client – no key needed for Open-Meteo
    client = httpx.AsyncClient(
        base_url="https://api.open-meteo.com",
        headers={
            "User-Agent": "MyApp/1.0 (+you@yourdomain.com)",
            "Accept":     "application/json",
        },
        timeout=30,
    )

    # ➌ Route maps → expose *only* GET /v1/forecast   (tool), exclude everything else
    route_maps = [
        RouteMap(
            methods=["GET"],
            pattern=r"^/v1/forecast/?(?:\?.*)?$",
            mcp_type=MCPType.TOOL,
        ),
        RouteMap(mcp_type=MCPType.EXCLUDE),
    ]

    # ➍ Create sub-MCP, mount under /weather
    sub = FastMCP.from_openapi(openapi_spec, client, name=name, route_maps=route_maps)
    parent.mount(name, sub)

    return [t.name for t in (await sub.get_tools()).values()]

